In [2]:
import fraud_msg_cls
import streamlit as st
from openai import OpenAI
import openai

openai.api_key = st.secrets["OPENAI_API_KEY"]
openai.base_url = st.secrets["OPENAI_BASE_URL"]
openai_model = st.secrets["OPENAI_MODEL"]

In [3]:
msg = "尊敬的用户，为响应国家政策，支付宝现推出降息优惠，请登录中国政府网生活号验证信息后，按照指引操作即可享受优惠。"

In [4]:
prediction = fraud_msg_cls.predict(msg)

d:\miniforge3\envs\DL\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [5]:
prediction

[('虚假信用服务类', 0.84811056),
 ('冒充电商物流客服类', 0.092546165),
 ('冒充公检法及政府机关类', 0.02540491),
 ('虚假购物、服务类', 0.013777527),
 ('无风险', 0.0071956175),
 ('网黑案件', 0.0050302274),
 ('冒充领导、熟人类', 0.0036921047),
 ('冒充军警购物类诈骗', 0.0018577599),
 ('虚假网络投资理财类', 0.0013425661),
 ('网络婚恋、交友类', 0.0010425716)]

In [7]:
prompt_template = """
我这里有一条疑似欺诈的信息，以下是信息内容：
{msg}

由我训练的模型得到，该短信属于的类别的各可能性为：
{prediction}

请你根据我训练的模型的预测结果，针对模型预测的可能性最大的的**一种类别**，给出针对收到短信的风险用户建议。

要求：
1. 不要输出模型预测的概率值
2. 可以针对短信中的内容的部分特征，结合模型预测的类别的典型特征，给出风险用户的建议。
3. 如果模型预测的类别的典型特征不足以给出建议，可以根据短信的内容给出建议。
4. 如果模型预测为无风险，可以恭喜用户，但也可以给出一些建议。
5. 只需要给出约 200 字的建议即可。建议有条理地列出。
6. 适量加入 emoji 表情，使得建议更加生动有趣。

建议内容：
"""

In [57]:
prompt = prompt_template.format(msg=msg, prediction=prediction)

client = OpenAI()

response = client.chat.completions.create(
  model=openai_model,
  messages=[
    {"role": "system", "content": "The following is a message that I received from a user and I need your help to respond to it."},
    {"role": "user", "content": prompt},
  ],
  max_tokens=1024,
  temperature=1.0,
)
print(response.choices[0].message.content)

根据模型预测，这条短信属于**虚假信用服务类**诈骗。以下是给您的建议：

1. **核实官方渠道** 🚨  
   支付宝不会通过短信要求用户登录其他网站验证信息。任何涉及“国家政策”“降息优惠”等话术，务必通过支付宝官方APP或客服电话（95188）核实。

2. **警惕陌生链接** 🔗  
   短信中的“中国政府网生活号”可能是伪造链接，切勿直接点击。政府机构不会通过私人账号推送金融操作指引。

3. **保护个人信息** 🔐  
   切勿在陌生页面输入银行卡号、密码或短信验证码！虚假信用服务常以“优惠”为诱饵，实为窃取账户信息。

4. **举报可疑内容** 📢  
   可将短信转发至支付宝举报入口（APP内“我的-帮助与反馈”）或反诈专线96110，帮助他人避免受骗。

记住：天下没有免费的午餐，优惠活动请认准官方公告！ 💡
